# Using LangChain Memory

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryMemory
from langchain.prompts import PromptTemplate

In [2]:
openai_api_key = 'dummy api key'
openai_api_base = 'http://localhost:8080/v1'

## ConversationBufferMemory

In [13]:
model = 'ggml-mpt-7b-instruct.bin'

memory = ConversationBufferMemory()
memory.save_context({"input": "Hi, I put an order for a laptop yesterday."}, {"output": "Hello, how can I assist you?"})

llm = ChatOpenAI(
    model = model,
    openai_api_key = openai_api_key,
    openai_api_base = openai_api_base,
    temperature = 0.0)

t ='The following is a friendly conversation between a human and an AI. \
AI is a customer service representative. AI should answer a question concisely. \
If the AI does not know the answer to a question, it truthfully says it \
does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:'

conversation = ConversationChain(
    llm = llm,
    memory = memory,
    prompt = PromptTemplate(input_variables=['history', 'input'], template=t),
    verbose = False)

In [14]:
print(memory.buffer)

Human: Hi, I put an order for a laptop yesterday.
AI: Hello, how can I assist you?


In [15]:
conversation.predict(input="Can you compute 3.14*2 for me?")

'Here you go : 6.28'

In [16]:
conversation.predict(input="What did I order?")

'A HP Pavilion x360 14-dh1000nia, 8 GB RAM and 256GB SSD'

In [17]:
print(memory.buffer)

Human: Hi, I put an order for a laptop yesterday.
AI: Hello, how can I assist you?
Human: Can you compute 3.14*2 for me?
AI: Here you go : 6.28
Human: What did I order?
AI: A HP Pavilion x360 14-dh1000nia, 8 GB RAM and 256GB SSD


## ConversationBufferWindowMemory

In [6]:
def print_history(memory):
    history = memory.load_memory_variables({})
    print(history['history'])

In [34]:
model = 'ggml-mpt-7b-instruct.bin'

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({"input": "Hi, I put an order for a laptop yesterday."}, {"output": "Hello, how can I assist you?"})

llm = ChatOpenAI(
    model = model,
    openai_api_key = openai_api_key,
    openai_api_base = openai_api_base,
    temperature = 0.0)

t ='The following is a friendly conversation between a human and an AI. \
AI is a customer service representative. AI should answer a question concisely. \
If the AI does not know the answer to a question, it truthfully says it \
does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:'

conversation = ConversationChain(
    llm = llm,
    memory = memory,
    prompt = PromptTemplate(input_variables=['history', 'input'], template=t),
    verbose = False)

In [35]:
print_history(memory)

Human: Hi, I put an order for a laptop yesterday.
AI: Hello, how can I assist you?


In [39]:
conversation.predict(input="Can you compute 3.14*2 for me? Please limit your answer to 1 word.")

'3.14 * 2 = 6.28'

In [40]:
print_history(memory)

Human: Can you compute 3.14*2 for me? Please limit your answer to 1 word.
AI: 3.14 * 2 = 6.28


In [41]:
conversation.predict(input="What did I order?")

'You ordered the following : a medium sized bag of potato chips, one small bottle water and a cheeseburger'

In [42]:
print_history(memory)

Human: What did I order?
AI: You ordered the following : a medium sized bag of potato chips, one small bottle water and a cheeseburger


## ConversationSummaryMemory

In [4]:
model = 'ggml-gpt4all-j-v1.3-groovy.bin'

llm = ChatOpenAI(
    model = model,
    openai_api_key = openai_api_key,
    openai_api_base = openai_api_base,
    temperature = 0.0)

memory = ConversationSummaryMemory(llm=llm)

memory.save_context({"input": "Hi, I put an order for a laptop yesterday."},
                    {"output": "Hello, how can I assist you?"})
memory.save_context({"input": "Can you compute 3.14*2 for me? Please limit your answer to 1 word."},
                    {"output": "3.14 * 2 = 6.28"})
memory.save_context({"input": "What did I order?"}, 
                    {"output": "A HP Pavilion x360 14-dh1000nia, 8 GB RAM and 256GB SSD"})

In [7]:
print_history(memory)

The AI offers to compute 3.14*2 for the human and limits its answer to 1 word. The AI offers assistance with the laptop order and continues to listen attentively until the human is satisfied with its assistance. The AI also offers to call the company for assistance if needed. The AI continues to assist the human with their laptop order and offers additional support if needed.


In [11]:
t ='The following is a friendly conversation between a human and an AI. \
AI is a customer service representative. AI should answer a question concisely. \
If the AI does not know the answer to a question, it truthfully says it \
does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:'

conversation = ConversationChain(
    llm = llm,
    memory = memory,
    prompt = PromptTemplate(input_variables=['history', 'input'], template=t),
    verbose = True)

In [12]:
conversation.predict(input='What was the brand name of the laptop I ordered?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. AI is a customer service representative. AI should answer a question concisely. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The AI offers to compute 3.14*2 for the human and limits its answer to 1 word. The AI offers assistance with the laptop order and continues to listen attentively until the human is satisfied with its assistance. The AI also offers to call the company for assistance if needed. The AI continues to assist the human with their laptop order and offers additional support if needed.
Human: What was the brand name of the laptop I ordered?
AI:

> Finished chain.


'The brand name of the laptop you ordered is Dell.'

In [13]:
print_history(memory)

The AI offers to compute 3.14*2 for the human and limits its answer to 1 word. The AI offers assistance with the laptop order and continues to listen attentively until the human is satisfied with its assistance. The AI also offers to call the company for assistance if needed. The AI continues to assist the human with their laptop order and offers additional support if needed.
